Approaches to representing text like `CountVectorizer` are sometimes called bag-of-words approaches, because they don't contain any information about the sequencing of the words. We're also using a fairly small dataset, so our model doesn't necessarily learn any general information about the mea

The next few notebooks will introduce more advanced techniques that try to address this problem. First, we'll take a look at word embeddings by focusing on the implementation of Word2Vec available in the `gensim` library.

In [0]:
import gensim
import pandas as pd
import numpy as np
import sklearn

In [2]:
from google.colab import drive

drive.mount('/content/gdrive')

train = pd.read_csv('gdrive/My Drive/RTANews_raw/arabic_train.csv')
val = pd.read_csv('gdrive/My Drive/RTANews_raw/arabic_val.csv')
test = pd.read_csv('gdrive/My Drive/RTANews_raw/arabic_test.csv')

train.head()

#Again, we'll limit ourselves to 20 classes for now.
train = train[train.label <= 20]
test = test[test.label <= 20]

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


We need to add a new preprocessing step here - tokenizing our text into individual words. `nltk`, which we used in a previous notebook for loading stopwords, has a nice utility for this. We're using a naive tokenizer that just looks for spaces between words, but `nltk` offers more advanced approaches as well, for example making use of regular expressions.

In [0]:
from nltk.tokenize import WhitespaceTokenizer

tokenizer = WhitespaceTokenizer()

train_words = [tokenizer.tokenize(t) for t in train.text]
test_words = [tokenizer.tokenize(t) for t in test.text]

Now we're going to train a set of word embeddings using the popular Word2Vec approach.

Word embeddings are high-dimensional vector representations of words. In previous notebooks, we were also providing vector representations of our text, but using sparse vectors, or vectors where most values are zero ("this word appears zero times in this document") and therefore don't contain any information.

Word2Vec is a specific approach for learning word embeddings with a shallow neural network. You can read more about it, and word embeddings in general, here: https://towardsdatascience.com/introduction-to-word-embedding-and-word2vec-652d0c2060fa.

We pass a few arguments when creating our model, to define the minimum number of times a word can appear to be included in the model and to use the skigram implementation of word2vec (the other being cbow, or continous bag of words, which does better for datasets larger than ours).

In [0]:
model = gensim.models.Word2Vec(train_words, min_count=1, sg=1)

Now we should be able to get the embedding for an indiviudal word out of our trained model. Let's give it a try:

In [6]:
model['المعارضة']

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


array([-0.4255591 ,  0.12782052, -0.81820583, -0.693488  ,  0.30725187,
       -0.04758458, -0.17393996, -0.34279114,  0.04506326, -0.06199343,
       -0.48297015,  0.6474886 , -0.20032613,  0.5090667 ,  0.49620962,
       -0.8726161 , -0.6187834 ,  0.728711  , -0.04358375,  0.34582934,
       -0.25561136,  0.62666553,  0.0953557 , -0.06873515, -1.0487658 ,
       -0.14294197, -0.38631955,  0.0282319 ,  0.16753352,  0.68459964,
       -0.29856402, -0.02244341,  0.3884174 ,  0.26702392, -0.01122629,
        0.345474  ,  0.18066911, -0.6465362 , -0.3510268 , -0.4122516 ,
       -0.48309022, -0.47709265, -0.5360168 , -0.78399575,  0.42255238,
       -0.04670966,  1.5143102 , -0.53281903, -0.03893476,  0.24694243,
       -0.01795327,  0.9169943 ,  0.06538728,  0.38306406,  0.58939254,
        0.11425997, -0.19075401, -0.01919077, -0.26314145, -0.15615922,
        0.2910486 , -0.18993138,  0.5362754 ,  0.03416961, -0.36792135,
        0.37644202,  0.95525354,  0.11944794, -0.4860445 ,  0.41

A brief tangent: one appealing characteristic of word embeddings is that similar words should be near to each other in embedding space. `gensim` allows us to test this by computing the similarity between two words and by returning the most similar words to a given word.

So let's give it a try!

In [7]:
model.similarity('الحرب', 'الصراع')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


0.681291

So the words 'war' and 'conflict' are fairly similar - makes sense.

Now, we'll look at the five words most similar to "opposition." This is also built into the `gensim` implementation of word2vec.

In [9]:
model.most_similar('المعارضة')[:5]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('المعتدلة', 0.8718295693397522),
 ('للمعارضة', 0.8214186429977417),
 ('الفصائل', 0.8155093193054199),
 ('معارضة', 0.808053731918335),
 ('المسلحة', 0.7812507152557373)]

This looks decent! Two of these are variants on the word opposition and the others are "armed," "factions," and "moderate" -- all words that Arab media outlets might use to describe an opposition movement.

We could further improve our results here by using a different metric, cosine similarity, to measure the distance between words in embedding space. But for now, let's return to our classification task.

Right now, we have vectors for individual words, but we want a single vector for each piece of text in our dataset. One way to solve this problem is by averaging the vectors of individual words. Let's create a simple function that does this.

In [0]:
#Remember that we've already tokenized our text, creating a 'list of lists.' Let's plan to feed that into our function.
def doc_vectorizer(text, model):
  doc_vec = 0
  count = 0

  if len(text) == 1:
    return model[text]

  for t in text:
    try:
      word_vec = model[t]
      doc_vec = doc_vec + word_vec
      count += 1
    except:
      pass
  
  doc_vec = doc_vec / count
  return doc_vec

In [26]:
X_train = [doc_vectorizer(t, model) for t in train_words]
X_test = [doc_vectorizer(t, model) for t in test_words]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # Remove the CWD from sys.path while we load stuff.


For similicity and for better comparison, let's feed our new vectors into the same `sklearn` classifier that we've used previously.

In [27]:
from sklearn.linear_model import LogisticRegression

Y_train = train.label
Y_test = test.label

classifier = LogisticRegression(max_iter = 5000, multi_class='multinomial').fit(X_train, Y_train)
preds = classifier.predict(X_test)
pd.crosstab(Y_test, preds)

col_0,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
label,,,,,,,,,,,,,,,,,,,,,
0,15,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,5,1,0,0,0
1,0,13,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,1,0,0,0
2,0,0,38,0,0,0,0,0,1,0,11,0,0,0,2,0,0,2,0,0,0
3,0,0,0,26,1,0,3,0,0,0,0,0,0,0,0,0,0,0,0,1,3
4,0,0,0,0,37,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
5,0,0,0,0,0,43,9,0,0,0,0,0,0,1,0,0,4,0,0,1,0
6,0,0,0,1,0,4,289,2,0,0,0,0,25,7,0,0,4,0,0,2,0
7,0,0,0,1,0,0,5,41,0,0,0,0,0,0,0,0,0,0,0,4,1
8,0,0,3,0,0,0,0,0,54,0,0,0,0,0,5,0,0,0,0,0,0


In [31]:
from sklearn.metrics import f1_score

f1_score(Y_test, preds, average = 'weighted')

0.7666117199190055

This looks good. Notice that the model is getting confused between categories 6 and 12 less often. The F1 score is also slightly improved.

But wait! There's another approach we can try here. `gensim` also offers a doc2vec implementation, for cases like ours where we really want document-level and not a word-level vectors. We use a different function within `gensim` to tokenize our documents.

In [0]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(train.text)]
doc_model = Doc2Vec(documents, min_count=1)

In [0]:
X_train = [doc_model.infer_vector(t) for t in train.text]
X_test = [doc_model.infer_vector(t) for t in test.text]

In [34]:

classifier = LogisticRegression(max_iter = 5000, multi_class='multinomial').fit(X_train, Y_train)
preds = classifier.predict(X_test)
f1_score(Y_test, preds, average = 'weighted')

0.42578872232835935

These results are much worse! That's probably due to the size of our training set. With more data, doc2vec would likely outperform word2vec for this kind of problem.

In the next notebook, we'll explore how to take advantage of large training sets when we don't have access to them ourselves.